## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap
import googlemaps

## Data Read in

In [7]:
import pandas as pd
from io import StringIO

csv_data = """Address,# of Units,Vintage
2160 Bolton Street,71,1931
2165 Bolton Street,66,1935
2180 Bolton Street,71,1931
3073 Buhre Ave *,46,1935
2060 Anthony Ave,68,1942
2140 Cruger Ave *,86,1930
2780 University Ave *,101,1949
3920 Bronx Blvd,49,1957
120 West 183rd Street,58,1936
222 Bedford Park Blvd,63,1939
250 Bedford Park Blvd,50,1937
3155 Grand Concourse,65,1936
655 East 230th Street *,54,1940
2160 Holland Ave,86,1930
221 East 201st Street,29,1931
275 East 201st Street,60,1940
210 East 201st Street *,29,1936
2966 Briggs Ave,42,1932
2100-2110 Bronx Park East *,157,1929
100 E Mosholu Prkway South,50,1928
316 E Mosholu Prkway South,54,1950
3565 Bruckner Blvd *,49,1931
3070 Hull Ave,50,1939
3339 Hull Ave,48,1940
55 East 210th Street *,47,1940
3001 Valentine Ave,36,1926
3010 Valentine Ave,42,1940
3013 Valentine Ave,49,1948
4305 Furman Ave *,43,1928
2559 Sedgwick Ave,57,1949
2575 Sedgwick Ave,86,1952
3881 Sedgwick Ave *,60,1929
33-35 East 208th Street *,56,1925
19 East Van Cortlandt Ave,43,1932
"""
df = pd.read_csv(StringIO(csv_data))


In [8]:
df

,Address,# of Units,Vintage
0,2160 Bolton Street,71,1931
1,2165 Bolton Street,66,1935
2,2180 Bolton Street,71,1931
3,3073 Buhre Ave *,46,1935
4,2060 Anthony Ave,68,1942
5,2140 Cruger Ave *,86,1930
6,2780 University Ave *,101,1949
7,3920 Bronx Blvd,49,1957
8,120 West 183rd Street,58,1936
9,222 Bedford Park Blvd,63,1939


## Get Coordinates

In [9]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

%store -r map_box_api_key

In [10]:
df['full_Address'] = df['Address'] + ' New York, NY'

In [11]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_Address'].apply(geocode)

In [12]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [17]:
columns_to_keep = [
    'Address', '# of Units', 'Vintage', 'full_Address', 'geocoded', 'lat',
       'lon'
]

map_df = df[columns_to_keep]

In [18]:
map_df.columns

Index(['Address', '# of Units', 'Vintage', 'full_Address', 'geocoded', 'lat',
       'lon'],
      dtype='object')

In [26]:
# 1) Popup HTML
def create_popup(row):
    lines = [
        f"<h3>{row['Address']}</h3>",
        f"<strong># of Units:</strong> {row['# of Units']}<br>",
        f"<strong>Vintage:</strong> {row['Vintage']}<br>"
    ]
    return "<div class='popup-content'>\n" + "\n".join(lines) + "\n</div>"

# 2) Initialize map
first_lat = float(map_df['lat'].iloc[0])
first_lon = float(map_df['lon'].iloc[0])
m = folium.Map(
    location=[first_lat, first_lon],
    zoom_start=12,
    scrollWheelZoom=False
)

# 3) Mapbox tile layer
folium.TileLayer(
    tiles=(
        f"https://api.mapbox.com/styles/v1/"
        f"mapbox/streets-v11/tiles/256/{{z}}/{{x}}/{{y}}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox",
    name="Streets",
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# 4) Optional CSS & Title
custom_css = """
<style>
  .popup-content { min-width:300px; font-size:14px; color:#333; }
  .leaflet-popup-content-wrapper { background:#f9f9f9; border:1px solid #bbb; }
  .leaflet-popup-tip { background:#f9f9f9; }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

title_html = """
<h3 style="text-align:center;
           font-family:Arial,sans-serif;
           font-size:18px;
           color:#333;
           margin-top:10px;">
  <b>Northern Bronx Portfolio</b>
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

# 5) Add controls
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# 6) Fixed radius for all bubbles
fixed_radius = 8  # pixels; tweak as needed

# 7) Add CircleMarkers
for _, row in map_df.iterrows():
    lat, lon = float(row['lat']), float(row['lon'])
    popup = folium.Popup(create_popup(row), max_width=300)

    folium.CircleMarker(
        location=[lat, lon],
        radius=fixed_radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        weight=1,
        popup=popup
    ).add_to(m)

# 8) Display
m  # in Jupyter this will render the interactive map

In [27]:
m.save('index.html')

In [28]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/NewYorkCity_05_20_25
